In [5]:
import pandas as pd 
import numpy as np 
import re

In [6]:
 patients = pd.read_csv('patients.csv')
 treatments = pd.read_csv('treatments.csv')
 adverse_reactions = pd.read_csv('adverse_reactions.csv')
 treatments_cut = pd.read_csv('treatments_cut.csv')

In [7]:
patients.head()

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1


In [8]:
treatments.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,veronika,jindrová,41u - 48u,-,7.63,7.20,NaN
1,elliot,richardson,-,40u - 45u,7.56,7.09,0.97
2,yukitaka,takenaka,-,39u - 36u,7.68,7.25,NaN
3,skye,gormanston,33u - 36u,-,7.97,7.62,0.35
4,alissa,montez,-,33u - 29u,7.78,7.46,0.32


In [9]:
treatments_cut.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,jožka,resanovič,22u - 30u,-,7.56,7.22,0.34
1,inunnguaq,heilmann,57u - 67u,-,7.85,7.45,NaN
2,alwin,svensson,36u - 39u,-,7.78,7.34,NaN
3,thể,lương,-,61u - 64u,7.64,7.22,0.92
4,amanda,ribeiro,36u - 44u,-,7.85,7.47,0.38


In [10]:
adverse_reactions.head()

,given_name,surname,adverse_reaction
0,berta,napolitani,injection site discomfort
1,lena,baer,hypoglycemia
2,joseph,day,hypoglycemia
3,flavia,fiorentino,cough
4,manouck,wubbels,throat irritation


In [11]:
#export data for manual assesmen t 
with pd.ExcelWriter('clinical_trails.xlsx') as writer: 
    patients.to_excel(writer,sheet_name="patients")
    treatments.to_excel(writer,sheet_name="treatments")
    treatments_cut.to_excel(writer,sheet_name="treatment_cut")
    adverse_reactions.to_excel(writer,sheet_name="adverse_reactions")

#### ISSUES with the dataset 
1) Dirty Data
    1) Table -`patients`
    -patient_id = 9 has misspelled name 'Dsvid' instead of David `accuracy`
    - state col sometimes contain full name and sometimes abbreviation `consistency`
    - data missing for 12 columns in address,city,state,sip_code,country,contact `completion`
    - zip code col has entries with 4 digit `validity` 
    - incorrect datatypes assigned , sex column,zip-code,birthdate `validity` 
    - duplicated data john doe in patients `accuracy`
    - one patients has weight = 48 pounds `validity`
    - one patients has height of 2.2 feet with weight 80+ kg `validity` 
    2) Table - `Treatments` & `Treatments-cuts`
     - given name and surname col is in all lower case `consistency`
     - remove u from auralin and novodra column `validity`
     - (-) in novadra and auralin column treated as nan `validity`
     - missing values in hba1c_change col `completion`
     - one duplicate entry in the table treatment named joseph day `accuracy`
     - hba1c_change , 9 instead of 4 `accuracy` 
     3) Table - `Adverse Reactions`
        - given name and surname is all in lowercase `consistency` 
2) Messy Data
    Table - `Patients`
      - contact col contains both phone and email `validity`
    
    table - `Treatments` & `Treatments-cuts`
      - Auralin and Novadra col should be split into 2 cols stand and end dose `consistency`

      - merge both the treatments and treatments-cuts table    
    table - `Adverse Reactions`
        - This table should not exists independently `validity`

In [12]:
patients[patients.duplicated(subset=['given_name','surname'])]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
229,230,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4
237,238,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4
244,245,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4
251,252,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4
277,278,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4


In [13]:
treatments[treatments.duplicated()]

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
136,joseph,day,29u - 36u,-,7.7,7.19,NaN


In [14]:
treatments[treatments.duplicated(subset=['given_name','surname'])]

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
136,joseph,day,29u - 36u,-,7.7,7.19,NaN


In [15]:
treatments_cut[treatments_cut.duplicated(subset=['given_name','surname'])]

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change


In [16]:
adverse_reactions.duplicated().sum()

0

In [17]:
patients.describe()

,patient_id,zip_code,weight,height,bmi
count,503.000000,491.000000,503.000000,503.000000,503.000000
mean,252.000000,49084.118126,173.434990,66.634195,27.483897
std,145.347859,30265.807442,33.916741,4.411297,5.276438
min,1.000000,1002.000000,48.800000,27.000000,17.100000
25%,126.500000,21920.500000,149.300000,63.000000,23.300000
50%,252.000000,48057.000000,175.300000,67.000000,27.200000
75%,377.500000,75679.000000,199.500000,70.000000,31.750000
max,503.000000,99701.000000,255.900000,79.000000,37.700000


In [18]:
patients[patients['weight'] == 48.800]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
210,211,female,Camilla,Zaitseva,4689 Briarhill Lane,Wooster,OH,44691.0,United States,330-202-2145CamillaZaitseva@superrito.com,11/26/1938,48.8,63,19.1


In [19]:
patients[patients['height'] == 27]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1


In [20]:
treatments.describe()

,hba1c_start,hba1c_end,hba1c_change
count,280.000000,280.000000,171.000000
mean,7.985929,7.589286,0.546023
std,0.568638,0.569672,0.279555
min,7.500000,7.010000,0.200000
25%,7.660000,7.270000,0.340000
50%,7.800000,7.420000,0.380000
75%,7.970000,7.570000,0.920000
max,9.950000,9.580000,0.990000


In [21]:
treatments.sort_values('hba1c_end')

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
39,krisztina,magyar,-,32u - 27u,7.52,7.01,0.51
80,hideki,haraguchi,-,37u - 35u,7.59,7.05,0.54
187,león,reynoso,-,38u - 40u,7.59,7.06,0.53
53,nasser,mansour,-,33u - 31u,7.51,7.06,0.95
86,ananías,enríquez,-,44u - 45u,7.58,7.07,0.51
...,...,...,...,...,...,...,...
192,valur,bjarkason,-,31u - 36u,9.71,9.41,0.30
171,justyna,kowalczyk,24u - 34u,-,9.84,9.44,NaN
75,mackenzie,mckay,-,44u - 43u,9.87,9.48,0.39
81,robert,wagner,43u - 49u,-,9.84,9.52,0.32


<h2>Accessing Data is an iterative Process</h2>

<h3>Data Quality Dimensions</h3> 
1)  completeness -> is data missin ? 
2) validity -> is data invalid -> negetive height -> duplicate patientid
3) accuracy --> data is valid but not accurate --> weight -> 1kg
4) consistency -> both valid and accurate but written differently -> new york and NY 

<h3>Order of Severity</h3>
Completeness <-- validity <-- Accuracy <- Consistency

### Data Cleaning Order

1) Quality -> completeness
2) Tidiness
3) Quality -> Validity
4) Quality -> Accuracy
5) Quality -> Consistency

### Steps involved in Data Cleaning 
1) Define 
2) Code
3) Test

`Always make sure to create a copy of your pandas dataframe before you start the cleaning process.`

In [22]:
patients_df = patients.copy()
treatments_df = treatments.copy()
treatments_cut_df = treatments_cut.copy()
adverse_reactions_df = adverse_reactions.copy()

### Define 
- replace all the mission values of patients df with no data
- sub hba1c_start from hba1c_start from change values
- in patients table we will use regex to separate email and phone number
- merging treatments and treatments_cut

In [23]:
# patients_df[patients_df['address'].isnull()]
#code
patients_df.fillna("No Data",inplace=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_26648\2036217908.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'No Data' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  patients_df.fillna("No Data",inplace=True)


In [24]:
#test
patients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   patient_id    503 non-null    int64  
 1   assigned_sex  503 non-null    object 
 2   given_name    503 non-null    object 
 3   surname       503 non-null    object 
 4   address       503 non-null    object 
 5   city          503 non-null    object 
 6   state         503 non-null    object 
 7   zip_code      503 non-null    object 
 8   country       503 non-null    object 
 9   contact       503 non-null    object 
 10  birthdate     503 non-null    object 
 11  weight        503 non-null    float64
 12  height        503 non-null    int64  
 13  bmi           503 non-null    float64
dtypes: float64(2), int64(2), object(10)
memory usage: 55.1+ KB


In [25]:
#code
treatments_df['hba1c_change'] = treatments_df['hba1c_start'] - treatments_df['hba1c_end']
treatments_cut_df['hba1c_change'] = treatments_cut_df['hba1c_start'] - treatments_cut_df['hba1c_end']

In [26]:
treatments_cut_df.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,jožka,resanovič,22u - 30u,-,7.56,7.22,0.34
1,inunnguaq,heilmann,57u - 67u,-,7.85,7.45,0.40
2,alwin,svensson,36u - 39u,-,7.78,7.34,0.44
3,thể,lương,-,61u - 64u,7.64,7.22,0.42
4,amanda,ribeiro,36u - 44u,-,7.85,7.47,0.38


In [27]:
patients_df['contact'].head(5)

0          951-719-9170ZoeWellish@superrito.com
1         PamelaSHill@cuvox.de+1 (217) 569-3204
2              402-363-6804JaeMDebord@gustr.com
3    PhanBaLiem@jourrapide.com+1 (732) 636-8246
4               334-515-7487TimNeudorf@cuvox.de
Name: contact, dtype: object

In [28]:

def find_contact_details(text: str) -> tuple:
    # If the value is NaN, return NaN for both phone and email
    if pd.isna(text):
        return np.nan, np.nan
    
    # Create the phone number pattern
    phone_number_pattern = re.compile(r"(\+[\d]{1,3}\s?)?(\(?[\d]{3}\)?\s?-?[\d]{3}\s?-?[\d]{4})")
    
    # Find the phone number from the text
    phone_number_match = re.findall(phone_number_pattern, text)
    
    # If no phone number is found, set phone_number to NaN
    if len(phone_number_match) == 0:
        phone_number = np.nan
    else:
        # Extract the phone number (with or without country code)
        phone_number = phone_number_match[0][1]  # The second group contains the actual phone number
    
    # Remove the phone number from the text to get the possible email address
    possible_email_add = re.sub(phone_number_pattern, "", text).strip()
    
    # If the remaining text is empty, set email to NaN
    if possible_email_add == "":
        possible_email_add = np.nan
    
    # Return the phone number and the possible email address
    return phone_number, possible_email_add

In [29]:
phone_number = patients_df["contact"].apply(lambda x : find_contact_details(x)[0])
email = patients_df["contact"].apply(lambda x : find_contact_details(x)[1])
patients_df['phone_number'] = phone_number
patients_df["email"] = email



AttributeError: module 'numpy' has no attribute 'matrix'

In [ ]:
patients_df.head()

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi,phone_number,email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6,951-719-9170,ZoeWellish@superrito.com
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2,(217) 569-3204,PamelaSHill@cuvox.de
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8,402-363-6804,JaeMDebord@gustr.com
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7,(732) 636-8246,PhanBaLiem@jourrapide.com
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1,334-515-7487,TimNeudorf@cuvox.de


In [ ]:
treatments_df = pd.concat([treatments_df,treatments_cut_df])

In [ ]:
treatments_df = treatments_df.melt(id_vars=['given_name','surname','hba1c_start','hba1c_end','hba1c_change'],var_name="type",value_name="dosage_range")

In [ ]:
treatments_df[treatments_df["given_name"] == 'veronika']

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,type,dosage_range
0,veronika,jindrová,7.63,7.2,0.43,auralin,41u - 48u
350,veronika,jindrová,7.63,7.2,0.43,novodra,-


In [ ]:
treatments_df = treatments_df[treatments_df['dosage_range'] != "-"]

In [ ]:
treatments_df['dosage_start'] = treatments_df['dosage_range'].str.split('-').str.get(0)
treatments_df['dosage_end'] = treatments_df['dosage_range'].str.split('-').str.get(1)

C:\Users\DELL\AppData\Local\Temp\ipykernel_21996\2601478946.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treatments_df['dosage_start'] = treatments_df['dosage_range'].str.split('-').str.get(0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_21996\2601478946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treatments_df['dosage_end'] = treatments_df['dosage_range'].str.split('-').str.get(1)


In [ ]:
treatments_df.drop(columns="dosage_range",inplace=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_21996\4293938020.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treatments_df.drop(columns="dosage_range",inplace=True)


In [ ]:
treatments_df

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,type,dosage_start,dosage_end
0,veronika,jindrová,7.63,7.20,0.43,auralin,41u,48u
3,skye,gormanston,7.97,7.62,0.35,auralin,33u,36u
6,sophia,haugen,7.65,7.27,0.38,auralin,37u,42u
7,eddie,archer,7.89,7.55,0.34,auralin,31u,38u
9,asia,woźniak,7.76,7.37,0.39,auralin,30u,36u
...,...,...,...,...,...,...,...,...
688,christopher,woodward,7.51,7.06,0.45,novodra,55u,51u
690,maret,sultygov,7.67,7.30,0.37,novodra,26u,23u
694,lixue,hsueh,9.21,8.80,0.41,novodra,22u,23u
696,jakob,jakobsen,7.96,7.51,0.45,novodra,28u,26u


In [ ]:
treatments_df['dosage_start'] = treatments_df['dosage_start'].str.replace('u','')
treatments_df['dosage_end']  = treatments_df['dosage_end'].str.replace('u','')

C:\Users\DELL\AppData\Local\Temp\ipykernel_21996\321745777.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treatments_df['dosage_start'] = treatments_df['dosage_start'].str.replace('u','')
C:\Users\DELL\AppData\Local\Temp\ipykernel_21996\321745777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treatments_df['dosage_end']  = treatments_df['dosage_end'].str.replace('u','')


In [ ]:
treatments_df["dosage_start"] = treatments_df["dosage_start"].astype('int')
treatments_df["dosage_end"] = treatments_df["dosage_end"].astype('int')

C:\Users\DELL\AppData\Local\Temp\ipykernel_21996\2175087033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treatments_df["dosage_start"] = treatments_df["dosage_start"].astype('int')
C:\Users\DELL\AppData\Local\Temp\ipykernel_21996\2175087033.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treatments_df["dosage_end"] = treatments_df["dosage_end"].astype('int')


In [ ]:
treatments_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 350 entries, 0 to 698
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   given_name    350 non-null    object 
 1   surname       350 non-null    object 
 2   hba1c_start   350 non-null    float64
 3   hba1c_end     350 non-null    float64
 4   hba1c_change  350 non-null    float64
 5   type          350 non-null    object 
 6   dosage_start  350 non-null    int32  
 7   dosage_end    350 non-null    int32  
dtypes: float64(3), int32(2), object(3)
memory usage: 21.9+ KB


In [ ]:
treatments_df = treatments_df.merge(adverse_reactions_df,how="left",on=['given_name','surname'])

In [ ]:
treatments_df

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,type,dosage_start,dosage_end,adverse_reaction
0,veronika,jindrová,7.63,7.20,0.43,auralin,41,48,NaN
1,skye,gormanston,7.97,7.62,0.35,auralin,33,36,NaN
2,sophia,haugen,7.65,7.27,0.38,auralin,37,42,NaN
3,eddie,archer,7.89,7.55,0.34,auralin,31,38,NaN
4,asia,woźniak,7.76,7.37,0.39,auralin,30,36,NaN
...,...,...,...,...,...,...,...,...,...
345,christopher,woodward,7.51,7.06,0.45,novodra,55,51,nausea
346,maret,sultygov,7.67,7.30,0.37,novodra,26,23,NaN
347,lixue,hsueh,9.21,8.80,0.41,novodra,22,23,injection site discomfort
348,jakob,jakobsen,7.96,7.51,0.45,novodra,28,26,hypoglycemia


In [ ]:
treatments_cut_df

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,jožka,resanovič,22u - 30u,-,7.56,7.22,0.34
1,inunnguaq,heilmann,57u - 67u,-,7.85,7.45,0.40
2,alwin,svensson,36u - 39u,-,7.78,7.34,0.44
3,thể,lương,-,61u - 64u,7.64,7.22,0.42
4,amanda,ribeiro,36u - 44u,-,7.85,7.47,0.38
...,...,...,...,...,...,...,...
65,rovzan,kishiev,32u - 37u,-,7.75,7.41,0.34
66,jakob,jakobsen,-,28u - 26u,7.96,7.51,0.45
67,bernd,schneider,48u - 56u,-,7.74,7.44,0.30
68,berta,napolitani,-,42u - 44u,7.68,7.21,0.47


In [ ]:
patients_df.loc[patients_df['patient_id'] == 9, 'given_name'] = "David"

In [ ]:
patients_df[patients_df['patient_id'] == 9]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi,phone_number,email
8,9,male,David,Gustafsson,1790 Nutter Street,Kansas City,MO,64105.0,United States,816-265-9578DavidGustafsson@armyspy.com,3/6/1937,163.9,66,26.5,816-265-9578,DavidGustafsson@armyspy.com


In [ ]:
patients_df.head()

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi,phone_number,email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6,951-719-9170,ZoeWellish@superrito.com
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2,(217) 569-3204,PamelaSHill@cuvox.de
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8,402-363-6804,JaeMDebord@gustr.com
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7,(732) 636-8246,PhanBaLiem@jourrapide.com
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1,334-515-7487,TimNeudorf@cuvox.de


In [ ]:
# patients_df['zip_code'] = patients_df['zip_code'].astype('int')
# [lambda x : x for i in patients_df['zip_code']] 
patients_df.loc[patients_df['zip_code'] == 'No Data'] = 0

In [ ]:
patients_df[patients_df['zip_code'] == 'No Data']

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi,phone_number,email


In [ ]:
patients_df['zip_code'] = patients_df['zip_code'].astype('int')

In [ ]:
patients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   patient_id    503 non-null    int64  
 1   assigned_sex  503 non-null    object 
 2   given_name    503 non-null    object 
 3   surname       503 non-null    object 
 4   address       503 non-null    object 
 5   city          503 non-null    object 
 6   state         503 non-null    object 
 7   zip_code      503 non-null    int32  
 8   country       503 non-null    object 
 9   contact       503 non-null    object 
 10  birthdate     503 non-null    object 
 11  weight        503 non-null    float64
 12  height        503 non-null    int64  
 13  bmi           503 non-null    float64
 14  phone_number  503 non-null    object 
 15  email         503 non-null    object 
dtypes: float64(2), int32(1), int64(2), object(11)
memory usage: 61.0+ KB


In [ ]:
# four_dig_counts = 0
# for i in patients_df['zip_code']:
#     i = str(i)
#     if len(i) == 4:
#         four_dig_counts+=1

# print(four_dig_counts)

49


In [ ]:
patients_df[patients_df.duplicated(subset=['patient_id'])]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi,phone_number,email
219,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0
230,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0
234,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0
242,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0
249,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0
257,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0
264,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0
269,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0
278,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0
286,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0


In [ ]:
patients_df = patients_df[patients_df['patient_id'] != 0]

In [ ]:
patients_df

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi,phone_number,email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6,951-719-9170,ZoeWellish@superrito.com
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2,(217) 569-3204,PamelaSHill@cuvox.de
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8,402-363-6804,JaeMDebord@gustr.com
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7,(732) 636-8246,PhanBaLiem@jourrapide.com
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1,334-515-7487,TimNeudorf@cuvox.de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,499,male,Mustafa,Lindström,2530 Victoria Court,Milton Mills,ME,3852,United States,207-477-0579MustafaLindstrom@jourrapide.com,4/10/1959,181.1,72,24.6,207-477-0579,MustafaLindstrom@jourrapide.com
499,500,male,Ruman,Bisliev,494 Clarksburg Park Road,Sedona,AZ,86341,United States,928-284-4492RumanBisliev@gustr.com,3/26/1948,239.6,70,34.4,928-284-4492,RumanBisliev@gustr.com
500,501,female,Jinke,de Keizer,649 Nutter Street,Overland Park,MO,64110,United States,816-223-6007JinkedeKeizer@teleworm.us,1/13/1971,171.2,67,26.8,816-223-6007,JinkedeKeizer@teleworm.us
501,502,female,Chidalu,Onyekaozulu,3652 Boone Crockett Lane,Seattle,WA,98109,United States,ChidaluOnyekaozulu@jourrapide.com1 360 443 2060,2/13/1952,176.9,67,27.7,360 443 2060,ChidaluOnyekaozulu@jourrapide.com1


In [ ]:
patients_df[patients_df.duplicated(subset=['given_name','surname'])]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi,phone_number,email
229,230,male,John,Doe,123 Main Street,New York,NY,12345,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4,1234567890,johndoe@email.com
237,238,male,John,Doe,123 Main Street,New York,NY,12345,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4,1234567890,johndoe@email.com
244,245,male,John,Doe,123 Main Street,New York,NY,12345,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4,1234567890,johndoe@email.com
251,252,male,John,Doe,123 Main Street,New York,NY,12345,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4,1234567890,johndoe@email.com
277,278,male,John,Doe,123 Main Street,New York,NY,12345,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4,1234567890,johndoe@email.com


In [ ]:
patients_df = patients_df.drop_duplicates(subset=['given_name','surname'],keep='last')

In [ ]:
patients_df[patients_df.duplicated(subset=['given_name','surname'])]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi,phone_number,email


In [30]:
patients_df

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,499,male,Mustafa,Lindström,2530 Victoria Court,Milton Mills,ME,3852.0,United States,207-477-0579MustafaLindstrom@jourrapide.com,4/10/1959,181.1,72,24.6
499,500,male,Ruman,Bisliev,494 Clarksburg Park Road,Sedona,AZ,86341.0,United States,928-284-4492RumanBisliev@gustr.com,3/26/1948,239.6,70,34.4
500,501,female,Jinke,de Keizer,649 Nutter Street,Overland Park,MO,64110.0,United States,816-223-6007JinkedeKeizer@teleworm.us,1/13/1971,171.2,67,26.8
501,502,female,Chidalu,Onyekaozulu,3652 Boone Crockett Lane,Seattle,WA,98109.0,United States,ChidaluOnyekaozulu@jourrapide.com1 360 443 2060,2/13/1952,176.9,67,27.7
